## Scraping module
Scraping of straits times news article by tag (food-prices) <br>
Hard-coded a bit

### Imports

In [1]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [2]:
#!pip install newspaper3k
#!pip install selenium
#!pip install webdriver-manager

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd
import datetime

In [4]:
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
from selenium.webdriver.common.by import By

### Newspaper Article Summariser

In [6]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    full_text = article.text
    location = full_text[0:full_text.find("-")]
    return article.summary, article.keywords, article.publish_date, location

### StraitsTimes Scraper

In [7]:
# change to ur own directory!
#driver = webdriver.Chrome(r"C:/Users/jonat/chromedriver.exe")
driver = webdriver.Chrome(ChromeDriverManager().install())

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"views-row")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)-1):
        news_details = []
        try: 
            xpath = f'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/div/div[{i+1}]/div/div/a'        
            url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
            summary = getSummary(url)
            date = summary[2].strftime('%d/%m/%Y')[3:]
            news_details.append(date)
            news_details.append(summary[3])
            news_details.append(all_news_per_page[i].text)
            news_details.append(url)
            news_details.append(summary[0])
            news_details.append(", ".join(summary[1]))
            output.append(news_details)
        except: 
            continue
    return output

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.21M/6.21M [00:00<00:00, 24.7MB/s]
C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
def getStraitsTimesNews(base_url):
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
#     filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within three years')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
#     nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/nav/ul/li[2]/a')))[0]
#     if (nextpage.get_attribute('title') == 'Go to next page'):
#         nextpage.click()
# #         print('clicked next')
#         output.extend(getNews())
#     nextpage = True
#     while (nextpage):  
#         try:            
#             nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="content"]/section/div[4]/div/div/div/div/div/article/div/div/div/div[1]/div/div/nav/ul/li[3]/a')))[0]
#             if (nextpage.get_attribute('title') == 'Go to next page'):
#                 nextpage.click()
# #                 print('clicked next')
#                 output.extend(getNews())
#             else: 
#                 nextpage = False
#         except:
# #             print("no next page!")
#             nextpage = False
    return output

output = []
for i in range(0,10):
    print(i)
    url = f"https://www.straitstimes.com/tags/Bukit-Merah?page={i}"
    output.extend(getStraitsTimesNews(url))
#     print(output)

# convert to df and export as csv
df = pd.DataFrame(output, columns=['date','location','news title', 'news source(url)', 'content summary', 'keywords'])
#df.to_csv(f"Ang-Mo-Kio-tag_straitstimes.csv")
#df.to_csv(r'Bedok-tag_straitstimes.csv')

0


C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
C:\Users\jonat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  


TimeoutException: Message: 


In [ ]:
df.to_csv(r'C:/Users/jonat/Bukit-Merah-tag_straitstimes.csv')